In [ ]:
!pip install --user wrf-python

In [1]:
from __future__ import print_function

# This jupyter notebook command inserts matplotlib graphics in 
# to the workbook
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

# Make sure the environment is good
import numpy as np
import cartopy
from cartopy import crs
from cartopy.feature import NaturalEarthFeature
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from xarray import DataArray
import os
from glob import glob

from wrf import (getvar, interplevel, vertcross, 
                 vinterp, CoordPair, ALL_TIMES, to_np,
                 xy_to_ll, ll_to_xy, get_cartopy, latlon_coords,
                 cartopy_xlim, cartopy_ylim)

filename_regex = \
    '/explore/nobackup/projects/ilab/projects/LobodaTFO/software/' + \
    'WRF/run/wrfout_d02_*'
wrf_files = glob(filename_regex)

ModuleNotFoundError: No module named 'wrf'

In [ ]:
# Import animation specific Python packages
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Use multi-file variable 'wrf_files' for this example
# Get sea level pressure for all times
slp_all = getvar(wrf_files, "slp", timeidx=ALL_TIMES)
# Use ALL_TIMES to make sure every file and all data is selected from all three wrfout files. 

# Get the cartopy projection object
cart_proj = get_cartopy(slp_all)

# Create figure and ax object
fig = plt.figure(figsize=(10,7.5))
ax_slp = plt.axes(projection=cart_proj)

# Download and create the states, land, and oceans using cartopy features
states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none',
                                      name='admin_1_states_provinces_shp')
land = NaturalEarthFeature(category='physical', name='land', scale='50m',
                                    facecolor='green')
ocean = NaturalEarthFeature(category='physical', name='ocean', scale='50m',
                                     facecolor='blue')

slp_levels = np.arange(950.,1030.,5.)

num_frames = slp_all.shape[0] 
# Look at shape because we read in all 9 time steps across the three files. Selected [0] to selected for the Time
# dimension in the array 

# This function is called for each frame of the animation, where
# i is the frame index. Here is where the animation frames need 
# to be created.
def animate(i):
    ax_slp.clear()
    slp = slp_all[i,:]
    
    # Get the lat/lon coordinates
    lats, lons = latlon_coords(slp)
    
    ax_slp.add_feature(ocean)
    ax_slp.add_feature(land)
    ax_slp.add_feature(states, linewidth=.5, edgecolor="black")
    
    slp_contours = ax_slp.contour(to_np(lons), 
                                  to_np(lats), 
                                  to_np(slp), 
                                  levels=slp_levels,
                                  colors="black", 
                                  zorder=5,
                                  transform=crs.PlateCarree()) 
    
    # Add contour labels for pressure
    ax_slp.clabel(slp_contours, fmt="%i")
    
    # Set the map bounds
    ax_slp.set_xlim(cartopy_xlim(slp))
    ax_slp.set_ylim(cartopy_ylim(slp))
        
    return ax_slp

# Create the animation by supplying a figure, the animation object, 
# the number of frames and an interval in milliseconds that is the 
# delay between frames.
ani = FuncAnimation(fig, animate, num_frames, interval=500)

# To work with jupyter notebook, you need to use the HTML generated
# by the HTML function in the IPython.display package.
# If you change 'to_jshtml' to be 'to_html5_video', you will get an 
# HTML5 video instead.

#HTML(ani.to_html5_video())
HTML(ani.to_jshtml())